#### 介绍

In [1]:
'''
输入待爬取的内容、专辑数量、页数，即可开始自动爬取音频。音频来自喜马拉雅
'''

'\n输入待爬取的内容、专辑数量、页数，即可开始自动爬取音频。音频来自喜马拉雅\n'

#### 导入模块

In [2]:
import requests
from lxml import etree
import re,os
from tqdm import tqdm

#### 功能函数

In [3]:
class get_audio:
    def __init__(self):
        # 伪装成chrome浏览器进行网页登录
        self.headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'
                      
                       }
        # 文件保存地址
        self.dirs_save = './爬取的文件/'

    def check_name(self,s_name):
        """处理文件名非法字符的方法"""
        s_name = s_name.replace(' ','-')
        pattern = re.compile(r"[\/\\\:\*\?\"\<\>\|]")  # '/ \ : * ? " < > |'
        new_name = re.sub(pattern, "_", s_name)  # 替换为下划线
        return new_name 
    
    # 根据专辑与页数，生成对应网址，并返回该网页的解析
    def get_urlpage_s(self,url_album,page):
        # 某专辑第一页的网址，从中可获取专辑页数，f'https://www.ximalaya.com{album}'
        url_page = f'https://www.ximalaya.com{url_album}p{page}/'
        print(url_page)
        response = requests.get(url=url_page, headers=self.headers)
        html_data = response.text
        selector = etree.HTML(html_data)
        return selector
    
    # 输入要搜索的对象、专辑数量、要下载的专辑内页数
    def search_name(self,content_to_search):
        search_url = f'https://www.ximalaya.com//search/{content_to_search}'
        response = requests.get(url=search_url, headers=self.headers)
        html_data = response.text
        selector = etree.HTML(html_data)
        # 搜索获得专辑网址列表
        url_album_list = selector.xpath('//*[@class="xm-album"]/div/a/@href')

        return url_album_list
    
    # 获取该专辑内音频页数
    def get_pagesnum(self,selector):
        pages_to_search = selector.xpath('//*[@class="page-link WJ_"]/@href')
        if len(pages_to_search) > 1:
            pages_num = len(pages_to_search) - 1
        else:
            pages_num = 1
        return pages_num
    
    # 获得音频下载地址
    def get_m4a_url(self,href):
        m4a_id = href.split('/')[-1]
        json_url = f'https://www.ximalaya.com/revision/play/v1/audio?id={m4a_id}&ptype=1'
        json_data = requests.get(url=json_url, headers=self.headers).json()
        m4a_url = json_data['data']['src']        
        return m4a_url
    
    # 下载文件
    def download_file(self,url, file_name, headers):
        r = requests.get(url,headers=self.headers, stream=True)
        chunk_size = 1024*1024
        content_size = int(r.headers['content-length'])/1024/1024  
        print('开始下载...大小为：{:.2f}MB'.format(content_size))
        f = open(file_name, "wb")
        for chunk in tqdm(r.iter_content(chunk_size=chunk_size)) :
            f.write(chunk)
        print(f'{file_name}文件保存完毕...')
        return content_size    

#### 爬取的内容、专辑数量、页数

In [4]:
# 输入要爬取的内容
content_to_search = input('请输入要爬取的内容:')#search_content = '郭德纲'
# 输入爬取的专辑数量
albums_to_search = int(input('请输入爬取的专辑数量：'))
# 输入爬取的页数
pages_to_search = int(input('请输入爬取的页数：'))



请输入要爬取的内容:催眠
请输入爬取的专辑数量：1
请输入爬取的页数：1


#### 开始爬取

In [ ]:
# 已爬取的专辑数量
album_searched = 0
# 已爬取的页数
pages_searched = 0

crawler = get_audio()
# 创建待保存的文件路径
if not os.path.exists(crawler.dirs_save):
    os.makedirs(crawler.dirs_save)
url_album_list = crawler.search_name(content_to_search)

for url_album in url_album_list:
    print(url_album)
    # 根据页数生成对应网址 如https://www.ximalaya.com/xiangsheng/9723091/p3/
    for page_ in range(pages_to_search):
        page = page_ + 1 
        
        selector = crawler.get_urlpage_s(url_album,page)
        
        # 该专辑最大页数
        max_pages = crawler.get_pagesnum(selector)
        
        # 读取该页所有音频的标题与下载相关地址
        s_titles = selector.xpath('//*[@class="text lF_"]/a/@title')
        s_hrefs = selector.xpath('//*[@class="text lF_"]/a/@href')
        
        # 下载该页内所有音频
        for title,href in zip(s_titles,s_hrefs):
            # 
            file_name = crawler.dirs_save + crawler.check_name(title) + '.m4a'
            # 获得音频下载地址
            m4a_url = crawler.get_m4a_url(href)
            # 下载音频
            crawler.download_file(m4a_url, file_name, crawler.headers)
            #print(f'下载{title}')
        
        # 已爬取到最后一页，退出
        if page==max_pages:
            break
        
        pages_searched += 1
        if pages_searched >= pages_to_search:
            break
    
    album_searched += 1
    if album_searched >= albums_to_search:
        break
        
    print('下载结束')

/yinyue/2791923/
https://www.ximalaya.com/yinyue/2791923/p1/


0it [00:00, ?it/s]

开始下载...大小为：2.35MB


3it [00:02,  1.25it/s]


./爬取的文件/亭子下的雨声（5min).m4a文件保存完毕...


...

3it [00:01,  2.80it/s]


./爬取的文件/台风来时-大自然的声音-纯音乐［声谷］.m4a文件保存完毕...


0it [00:00, ?it/s]

开始下载...大小为：4.00MB


3it [00:02,  1.52it/s]